# Análise e tratamento de dados de incidentes aereos ANAC

In [2]:
import pandas as pd
import zipfile
import os

Base de dados

In [3]:
# caminho  JSON
Arquivo = r"C:\Users\david\Desktop\ProjetoANAC\DiferentesFormatos\V_OCORRENCIA_AMPLA.json"
df =pd.read_json(Arquivo)
df.head(5)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Hora_da_Ocorrência,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,33947,2022722007,AZUL LINHAS AÉREAS BRASILEIRAS S.A.,Incidente,2022-08-11,05:31:00,PALMAS,TO,Norte,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,NaN,NaN,NaN,ERJ 190-200 LR,L2J,E195,50790.0,125.0,EMBRAER,verdadeiro
1,29123,202140534,AZUL,Incidente Grave,2021-08-06,16:59,CAXIAS DO SUL,RS,Sul,CONTATO ANORMAL COM A PISTA,...,NaN,NaN,NaN,ERJ 190-400,L2J,E295,61500.0,143.0,EMBRAER,verdadeiro
2,32189,2022293670,FIGUEIREDO AVIAÇÃO AGRÍCOLA EIRELI,Incidente Grave,2022-05-21,12:25,DOURADOS,MS,Centro-Oeste,OPERAÇÃO A BAIXA ALTITUDE,...,NaN,NaN,NaN,EMB-201,L1P,IPAN,1800.0,1.0,NEIVA,verdadeiro
3,32190,2022354545,FENIX PARTICIPACOES LTDA,Incidente,2022-02-12,15:30,CAMPINAS,SP,Sudeste,COLISÃO COM AVE,...,NaN,NaN,NaN,PC-12/47E,L1T,PC12,4740.0,10.0,PILATUS,verdadeiro
4,32726,2022125796,ANAPE EMPREENDIMENTOS IMOBILIÁRIOS-EIRELI,Incidente,2022-06-26,13:45,CAMPINAS,SP,Sudeste,COLISÃO NO SOLO,...,NaN,NaN,NaN,A109E,H2T,A109,3000.0,8.0,AGUSTA,verdadeiro


Funções para o tratamento da base

In [4]:
#mostrar todas as colunas
pd.set_option('display.max_columns', None)

#Verifica se há nulos (base)
def show_null(df):
    null_columns = (df.isnull().sum(axis=0)/len(df)).sort_values(ascending=False).index
    
    null_data = pd.concat([df.isnull().sum(axis=0), 
                           (df.isnull().sum(axis=0)/len(df)).sort_values(ascending=False), 
                           df.loc[:, df.columns.isin(list(null_columns))].dtypes], 
                          axis=1)
    
    null_data = null_data.rename(columns={0: '#', 
                                          1: '% null', 
                                          2: 'type'}).sort_values(ascending=False, 
                                                                  by='% null')
    
    return null_data

#trata os nulos
def tratar_colunas_NAN(df,var_NaN,lista_prenchimento):
    filtered_df = var_NaN[var_NaN['#'] > 0]
    colunas_com_null = filtered_df.index.tolist()
    for i in range(len(colunas_com_null)):
        df[colunas_com_null[i]] = df[colunas_com_null[i]].fillna(lista_prenchimento[i])
    return df

# Salva em formato parquet
def salvar_parquet(diretorio,nome_arquivo,df):
    caminho_parquet = diretorio+nome_arquivo+".parquet"
    df.to_parquet(caminho_parquet, index=False)
    print('Arquivo Salvo no diretório')
    
# traz as colunas especificas de um dataframe
def trazer_colunas(df, lista_colunas):
    df = df[lista_colunas]
    return df

#Função substituir valores de uma determinada coluna
def substituir_valores(df,coluna,valor_atual,valor_substituto):
    df[coluna] = df[coluna].replace(valor_atual,valor_substituto)

## Usando as funções criadas Acima

Salvando os dados em formato parquet, economia no armazenamento em nuvem, podendo ter uma redução acima de 70% comparado com JSON

In [6]:
#!pip install pyarrow

   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
    --------------------------------------- 0.3/25.9 MB 6.8 MB/s eta 0:00:04
   - -------------------------------------- 0.9/25.9 MB 9.9 MB/s eta 0:00:03
   -- ------------------------------------- 1.6/25.9 MB 11.4 MB/s eta 0:00:03
   --- ------------------------------------ 2.4/25.9 MB 12.7 MB/s eta 0:00:02
   ----- ---------------------------------- 3.3/25.9 MB 14.2 MB/s eta 0:00:02
   ------ --------------------------------- 4.3/25.9 MB 15.4 MB/s eta 0:00:02
   -------- ------------------------------- 5.4/25.9 MB 16.6 MB/s eta 0:00:02
   ---------- ----------------------------- 6.7/25.9 MB 17.9 MB/s eta 0:00:02
   ------------ --------------------------- 7.9/25.9 MB 18.6 MB/s eta 0:00:01
   -------------- ------------------------- 9.2/25.9 MB 19.6 MB/s eta 0:00:01
   ---------------- ----------------------- 10.5/25.9 MB 21.8 MB/s eta 0:00:01
   ------------------ --------------------- 11.8/25.9 MB 24.2 MB/s eta 0:

In [8]:
diretorio = r"C:\Users\david\Desktop\ProjetoANAC\DiferentesFormatos\\"
nome_arquivo = "ArquivoIncidentes"
salvar_parquet(diretorio,nome_arquivo,df)

Arquivo Salvo no diretório


Importando o arquivo no novo formato .parquet
- Pode ser usado no power bi, só precisa adicionar uma linha → import pandas as pd

In [9]:
Caminho = r"C:\Users\david\Desktop\ProjetoANAC\DiferentesFormatos\ArquivoIncidentes.parquet" 
df=pd.read_parquet(Caminho)

Verificando valores faltantes - NaN

In [10]:
show_null(df)

,#,% null,type
Tipo_de_Aerodromo,8936,0.731200,object
ICAO,8795,0.719663,object
Longitude,4003,0.327551,object
Latitude,3995,0.326896,object
Danos_a_Aeronave,3710,0.303576,object
Municipio,3696,0.302430,object
Hora_da_Ocorrência,3655,0.299075,object
Fase_da_Operacao,3377,0.276328,object
Tipo_de_Ocorrencia,3009,0.246216,object
Descricao_do_Tipo,3009,0.246216,object


Tratando os dados faltantes 
- É importante ferificar os tipos de dados e indicar o preenchimento conforme o tipo de dados
- Foi criado listas com tipos por estarem intercalados object e float 

In [11]:
var_NaN = show_null(df)
bloc01 = ["Não informado" for _ in range(10)]
bloco2 = [0 for _ in range(13)]
bloco3 = ["Não informado" for _ in range(3)]
bloco4 = [0 for _ in range(1)]
bloco5 = ["Não informado" for _ in range(4)]
bloco6 = [0 for _ in range(2)]
bloco7 = ["Não informado" for _ in range(8)]
lista_prenchimento = bloc01 + bloco2 + bloco3 + bloco4 + bloco5 + bloco6 + bloco7
lista_prenchimento

df = tratar_colunas_NAN(df,var_NaN,lista_prenchimento)

Escolhendo apenas as colunas da análise

In [95]:
df.columns

Index(['Numero_da_Ocorrencia', 'Numero_da_Ficha', 'Operador_Padronizado',
       'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia',
       'Hora_da_Ocorrência', 'Municipio', 'UF', 'Regiao', 'Descricao_do_Tipo',
       'ICAO', 'Latitude', 'Longitude', 'Tipo_de_Aerodromo', 'Historico',
       'Matricula', 'Categoria_da_Aeronave', 'Operador', 'Tipo_de_Ocorrencia',
       'Fase_da_Operacao', 'Operacao', 'Danos_a_Aeronave',
       'Aerodromo_de_Destino', 'Aerodromo_de_Origem',
       'Lesoes_Fatais_Tripulantes', 'Lesoes_Fatais_Passageiros',
       'Lesoes_Fatais_Terceiros', 'Lesoes_Graves_Tripulantes',
       'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros',
       'Lesoes_Leves_Tripulantes', 'Lesoes_Leves_Passageiros',
       'Lesoes_Leves_Terceiros', 'Ilesos_Tripulantes', 'Ilesos_Passageiros',
       'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Desconhecidas_Passageiros',
       'Lesoes_Desconhecidas_Terceiros', 'Modelo', 'CLS', 'Tipo_ICAO', 'PMD',
       'Numero_de_Assentos', 'Nom

In [12]:
lista_colunas = ['Numero_da_Ocorrencia', 'Numero_da_Ficha', 'Operador_Padronizado', 
                 'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia',   
                 'Municipio', 'UF', 'Regiao', 'Descricao_do_Tipo','Historico',
                 'Matricula', 'Categoria_da_Aeronave', 'Operador', 'Fase_da_Operacao', 
                 'Operacao', 'Danos_a_Aeronave','Lesoes_Fatais_Tripulantes', 
                 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros','Modelo','Nome_do_Fabricante'
                 ]
df = trazer_colunas(df, lista_colunas)

# Limpesa de dados não necessarios para a coluna

## Será feito em todas as colunas
### UF

In [15]:
contagem  = df['UF'].value_counts()
contagem 

UF
SP    2856
RJ    1032
MG    1002
PR     993
RS     630
MT     589
GO     572
PA     520
SC     454
BA     436
AM     426
MS     299
DF     294
PE     221
CE     182
MA     175
ES     163
RR     108
TO     101
AC      90
RO      81
PI      80
PB      43
RN      40
AL      36
SE      33
AP      20
Name: count, dtype: int64

In [14]:
df_filtrado = df.drop(df[df['UF'] == 'Exterior'].index)
df_filtrado = df_filtrado.drop(df_filtrado[df_filtrado['UF'] == 'Indeterminado'].index)
df_filtrado = df_filtrado.drop(df_filtrado[df_filtrado['UF'] == 'Não informado'].index)
df_filtrado['UF'].value_counts()
df = df_filtrado

### Classificacao_da_Ocorrencia

In [18]:
pd.set_option('display.max_rows', None) 
contagem  = df['Classificacao_da_Ocorrência'].value_counts()
display(contagem) 

Classificacao_da_Ocorrência
Incidente             4887
Acidente              2874
Ocorrência Anormal    2016
Incidente Grave       1241
Ocorrência de Solo     457
Name: count, dtype: int64

In [17]:
df_filtrado = df.drop(df[df['Classificacao_da_Ocorrência'] == 'Indeterminado'].index)
df_filtrado['Classificacao_da_Ocorrência'].value_counts()
df = df_filtrado

Municipio por ter aproximadamente 40% não informado eu mantive os dados para não ter um percentual alto de perda de análise

In [101]:
pd.set_option('display.max_rows', None) 
contagem  = df['Municipio'].value_counts()

O mesmo aplica para Descricao_do_Tipo, Historico, Fase_da_Operacao, Operacao, Danos_a_Aeronave, Modelo, Nome_do_Fabricante

In [102]:
pd.set_option('display.max_rows', None) 
contagem  = df['Descricao_do_Tipo'].value_counts()

In [103]:
pd.set_option('display.max_rows', None) 
contagem  = df['Historico'].value_counts()

Categoria_da_Aeronave

In [19]:
pd.set_option('display.max_rows', None) 
contagem  = df['Categoria_da_Aeronave'].value_counts()

In [20]:
coluna = 'Categoria_da_Aeronave'
valor_atual = 'ESTRANGEIRA'
valor_substituto = 'Estrangeira'
substituir_valores(df,coluna,valor_atual,valor_substituto)

In [21]:
df_filtrado = df.drop(df[df['Categoria_da_Aeronave'] == 'INDEFINIDA'].index)
df_filtrado = df.drop(df[df['Categoria_da_Aeronave'] == "Não informado"].index)
df_filtrado = df.drop(df[df['Categoria_da_Aeronave'] == 'Sem matrícula'].index)
df_filtrado['Categoria_da_Aeronave'].value_counts()
df = df_filtrado

Os dados que precisaram serem tratados estão acima, vamos salvar esses dados no formato parquet 

In [22]:
Diretorio = r"C:\Users\david\Desktop\ProjetoANAC\DiferentesFormatos\\"
Nome_Arquivo = "ArquivoIncidentes_tratado.parquet"
caminho_parquet = Diretorio+Nome_Arquivo
df.to_parquet(caminho_parquet, index=False)

Usar o Notebook abaixo no Power BI

In [23]:
import pandas as pd
Caminho = r"C:\Users\david\Desktop\ProjetoANAC\DiferentesFormatos\ArquivoIncidentes_tratado.parquet" 
df=pd.read_parquet(Caminho)
df.head(12)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,Historico,Matricula,Categoria_da_Aeronave,Operador,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Modelo,Nome_do_Fabricante
0,33947,2022722007,AZUL LINHAS AÉREAS BRASILEIRAS S.A.,Incidente,2022-08-11,PALMAS,TO,Norte,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,A aeronave decolou do aeródromo Marechal Cunha...,PRAYJ,TPR,AZUL LINHAS AÉREAS BRASILEIRAS S.A.,Cruzeiro,Voo Regular,Nenhum,0.0,0.0,0.0,ERJ 190-200 LR,EMBRAER
1,29123,202140534,AZUL,Incidente Grave,2021-08-06,CAXIAS DO SUL,RS,Sul,CONTATO ANORMAL COM A PISTA,A AERONAVE DECOLOU DO AERÓDROMO VIRACOPOS (SBK...,PRPJN,TPR,AZUL LINHAS AEREAS BRASILEIRAS S.A,Pouso,Voo Regular,Leve,0.0,0.0,0.0,ERJ 190-400,EMBRAER
2,32189,2022293670,FIGUEIREDO AVIAÇÃO AGRÍCOLA EIRELI,Incidente Grave,2022-05-21,DOURADOS,MS,Centro-Oeste,OPERAÇÃO A BAIXA ALTITUDE,A aeronave decolou do aeródromo Asas do MS (SS...,PTGGQ,S11,FIGUEIREDO AVIAÇÃO AGRÍCOLA EIRELI,Manobra,Operação Agrícola,Leve,0.0,0.0,0.0,EMB-201,NEIVA
3,32190,2022354545,FENIX PARTICIPACOES LTDA,Incidente,2022-02-12,CAMPINAS,SP,Sudeste,COLISÃO COM AVE,A aeronave decolou do Aeródromo Estadual de Ca...,PRVZE,TPP,FENIX PARTICIPACOES LTDA,Decolagem,Voo Privado,Leve,0.0,0.0,0.0,PC-12/47E,PILATUS
4,32726,2022125796,ANAPE EMPREENDIMENTOS IMOBILIÁRIOS-EIRELI,Incidente,2022-06-26,CAMPINAS,SP,Sudeste,COLISÃO NO SOLO,A aeronave decolou do heliponto Santuário Naci...,PRPIN,TPP,ANAPE EMPREENDIMENTOS IMOBILIÁRIOS-EIRELI,Estacionamento,Voo Privado,Leve,0.0,0.0,0.0,A109E,AGUSTA
5,33267,2022814112,LATAM Airlines,Incidente,2022-07-31,RIO DE JANEIRO,RJ,Sudeste,AERÓDROMO,A aeronave decolou do aeródromo Antônio Carlos...,CCCOH,Não informado,LATAM Airlines,Táxi,Voo Regular,Nenhum,0.0,0.0,0.0,Não informado,Não informado
6,35928,2023813602,SIMOL EMPREENDIMENTOS IMOBILIARIOS LTDA,Incidente,2023-01-12,TAUBATÉ,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,A aeronave decolou do Aeródromo Jacarepagua-Ro...,PRJLS,TPP,SIMOL EMPREENDIMENTOS IMOBILIARIOS LTDA,Cruzeiro,Voo Privado,Nenhum,0.0,0.0,0.0,PA-34-220T,PIPER AIRCRAFT
7,36149,2023689258,AZUL LINHAS AÉREAS BRASILEIRAS S.A.,Incidente,2023-01-13,RIO DE JANEIRO,RJ,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,A aeronave decolou do aeroporto Santa Genoveva...,PRYRS,TPR,AZUL LINHAS AÉREAS BRASILEIRAS S.A.,Descida,Voo Regular,Nenhum,0.0,0.0,0.0,A320-253N,AIRBUS S.A.S.
8,36206,2023101831,SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA E DE...,Acidente,2023-01-27,BELÉM,PA,Norte,PERDA DE CONTROLE NO SOLO,A aeronave decolou do Aeródromo Val de Cans - ...,PPBAM,D17,SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA E DE...,Pouso,Operação Policial,Substancial,0.0,0.0,0.0,AS 350 B3,HELIBRAS
9,36206,2023101831,SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA E DE...,Acidente,2023-01-27,BELÉM,PA,Norte,PERDA DE CONTROLE NO SOLO,A aeronave decolou do Aeródromo Val de Cans - ...,PPBAM,D17,SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA E DE...,Pouso,Operação Policial,Substancial,0.0,0.0,0.0,AS 350 B3,HELIBRAS
